In [4]:
import warnings
warnings.filterwarnings('ignore')

import warnings
import pandas as pd
import geopandas as gpd
from pathlib import Path

# Suprimir advertencias
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

states = gpd.read_file("Estados.geojson")
mun = gpd.read_file("mun_MX_SIMP.geojson")

def load_geodata(file_path):
    """Carga datos geoespaciales desde un archivo."""
    try:
        return gpd.read_file(file_path)
    except Exception as e:
        raise FileNotFoundError(f"Error al leer el archivo {file_path}: {e}")


def get_user_input():
    """Solicita entrada del usuario para estado y municipio."""
    estado = input("Indica el nombre del estado (ej: Aguascalientes, Sonora, Chihuahua): ").strip()
    municipio = input("Indica el nombre del municipio (ej: San José de Gracia, Tepezalá): ").strip()
    return estado, municipio


def filter_estado(estados_gdf, estado_nome):
    """Filtra y elimina el estado seleccionado de los datos."""
    estado_normalizado = estado_nome.lower()
    estado_encontrado = estados_gdf[
        estados_gdf["name"].str.strip().str.lower() == estado_normalizado
    ]

    if estado_encontrado.empty:
        raise ValueError(f"Estado '{estado_nome}' no encontrado.")

    otros_estados = estados_gdf[
        estados_gdf["name"].str.strip().str.lower() != estado_normalizado
    ]

    return otros_estados


def process_municipios(municipios_gdf, municipio_nome, estado_nome):
    """Procesa los municipios del estado seleccionado, separando el municipio de interés."""
    estado_normalizado = estado_nome.lower()
    municipio_normalizado = municipio_nome.lower()

    # Filtra solo los municipios del estado indicado
    municipios_do_estado = municipios_gdf[
        municipios_gdf["state"].str.strip().str.lower() == estado_normalizado
    ]

    if municipios_do_estado.empty:
        raise ValueError(f"Ningún municipio encontrado para el estado '{estado_nome}'.")

    # Verifica si el municipio existe en el estado
    municipio_selecionado = municipios_do_estado[
        municipios_do_estado["name"].str.strip().str.lower() == municipio_normalizado
    ]

    if municipio_selecionado.empty:
        raise ValueError(f"Municipio '{municipio_nome}' no encontrado en el estado '{estado_nome}'.")

    # Configura el municipio seleccionado
    municipio_selecionado = municipio_selecionado.copy()
    # Elimina el prefijo MUN- si existe, antes de añadir AOI-
    id_limpo = municipio_selecionado["id"].astype(str).str.replace("MUN-", "", regex=False)
    municipio_selecionado["id"] = "AOI-" + id_limpo
    municipio_selecionado["poly_type"] = "core"

    # Otros municipios del mismo estado (excepto el seleccionado)
    outros_municipios_estado = municipios_do_estado[
        municipios_do_estado["name"].str.strip().str.lower() != municipio_normalizado
    ]

    return municipio_selecionado, outros_municipios_estado


def create_project(outros_estados, outros_municipios_estado, municipio_selecionado):
    """Crea el proyecto final concatenando todas las capas."""
    return pd.concat([outros_estados, outros_municipios_estado, municipio_selecionado],
                    ignore_index=True)


def save_project(projeto_gdf, municipio_nome):
    """Guarda el proyecto en un archivo GeoJSON."""
    output_filename = f"{municipio_nome.replace(' ', '_')}_AOI.geojson"
    try:
        projeto_gdf.to_file(output_filename, driver='GeoJSON')
        print(f"Proyecto guardado con éxito en {output_filename}")
    except Exception as e:
        raise IOError(f"Error al guardar el proyecto: {e}")


def main():
    """Función principal del programa."""
    try:
        estado, municipio = get_user_input()

        
        estados = states.copy()  
        municipios = mun.copy()

        outros_estados = filter_estado(estados, estado)
        municipio_selecionado, outros_municipios_estado = process_municipios(municipios, municipio, estado)

        projeto = create_project(outros_estados, outros_municipios_estado, municipio_selecionado)
        save_project(projeto, municipio)

    except (FileNotFoundError, ValueError, IOError) as e:
        print(f"Error: {e}")
    except KeyboardInterrupt:
        print("\nOperación cancelada por el usuario.")
    except Exception as e:
        print(f"Error inesperado: {e}")


if __name__ == "__main__":
    main()

Indica el nombre del estado (ej: Aguascalientes, Sonora, Chihuahua):  Quintana Roo
Indica el nombre del municipio (ej: San José de Gracia, Tepezalá):  Benito Juárez


Proyecto guardado con éxito en Benito_Juárez_AOI.geojson


In [2]:
teste = gpd.read_file("Cozumel_AOI.geojson")

In [3]:
Benito Juárez	

,id,name,poly_type,state,geometry
0,PRO-1,Aguascalientes,periphery,None,"POLYGON ((-101.86167 22.02888, -101.86144 22.0..."
1,PRO-2,Baja California,periphery,None,"MULTIPOLYGON (((-114.81357 32.49391, -114.8135..."
2,PRO-3,Baja California Sur,periphery,None,"MULTIPOLYGON (((-112.75982 27.90325, -112.7602..."
3,PRO-4,Campeche,periphery,None,"MULTIPOLYGON (((-90.40452 20.84676, -90.40439 ..."
4,PRO-5,Coahuila de Zaragoza,periphery,None,"POLYGON ((-99.84367 27.76939, -99.845 27.77094..."
5,PRO-6,Colima,periphery,None,"MULTIPOLYGON (((-104.69023 19.17654, -104.6904..."
6,PRO-7,Chiapas,periphery,None,"MULTIPOLYGON (((-93.60794 17.31493, -93.62557 ..."
7,PRO-8,Chihuahua,periphery,None,"POLYGON ((-103.30696 29.00393, -103.30948 29.0..."
8,PRO-9,Ciudad de México,periphery,None,"POLYGON ((-98.979 19.07446, -98.9737 19.07955,..."
9,PRO-10,Durango,periphery,None,"POLYGON ((-103.69909 26.72526, -103.83814 26.7..."
